In [46]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Inicializa MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8)


def calculate_angle(a, b, c):
    """Calcula el ángulo entre tres puntos clave."""
    a = np.array([a.x, a.y])  # Primer punto
    b = np.array([b.x, b.y])  # Punto central (vértice)
    c = np.array([c.x, c.y])  # Tercer punto

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

class PressBancaCounter:
    def __init__(self):
        self.reps = 0
        self.state = "up"  # Estado inicial: en extensión completa

    def update(self, left_angle, right_angle):
        # Umbrales para los ángulos
        min_flex_angle = 80  # Punto máximo de flexión
        max_extend_angle = 100  # Punto máximo de extensión

        # Determinar el estado actual basado en los ángulos
        if left_angle >= max_extend_angle and right_angle >= max_extend_angle:
            if self.state == "down":
                self.reps += 1  # Contar repetición cuando se vuelva a extensión
                
            self.state = "up"
        elif left_angle <= min_flex_angle and right_angle <= min_flex_angle:
            self.state = "down"

        return self.reps
class Rm:
    def __init__(self, weight):
        self.rm = 0.0
        self.weight = weight   
           
    def Epley(self, reps):
        newReps = self.RepsToInt(reps)
        self.rm = (self.weight*newReps*0.033)+self.weight
        return self.rm
    
    def Brzycki(self, reps):
        newReps = self.RepsToInt(reps)
        self.rm = self.weight/(1.0278 - (0.0278*newReps))
        return self.rm
    
    def EpleyReducida(self, reps):
        newReps = self.RepsToInt(reps)
        work = 0.03
        suma = 0
        for i in range(5):
            suma += (self.weight*newReps*work)+self.weight
            work -= 0.001
        suma = suma/5
        self.rm = round(suma, 1)
        return self.rm
    
    def RepsToInt(self, reps):
        return float(reps.split(':')[-1].strip())
    
class VelocityCalculator:
    def __init__(self):
        self.last_time = None
        self.velocities = []

    
    def update(self, current_time):
        """
        Calcula y almacena la velocidad basada en el tiempo entre eventos.
        """
        if self.last_time is None:
            self.last_time = current_time
            return None  # No hay velocidad calculable en el primer evento

        tiempo_estimado = current_time - self.last_time
        self.last_time = current_time

        if tiempo_estimado > 0:  # Prevenir divisiones por cero
            velocidad = 1 / tiempo_estimado  # Velocidad proporcional al tiempo
            self.velocities.append(velocidad)
            return velocidad

        return None
    
    def get_average_velocity(self):
        """
        Calcula la velocidad media de todas las repeticiones registradas.
        """
        if self.velocities:
            return sum(self.velocities) / len(self.velocities)
        return 0.0

  
        
class PressBancaState: 
    def __init__(self):
        self.puntuacion_extension = 0.0
        self.puntuacion_bajo = 0.0
        self.puntuacion_rom = 0.0
        self.repeticion = 0
        self.state = "up"  # Estado inicial (extensión máxima)
        self.ROM = []

    def calculate_extension_score(self, left_angle, right_angle):
        """Calcula la puntuación de la extensión en el punto máximo."""
        def score_angle(angle):
            if 150 <= angle <= 160:
                return 9 + (angle - 150) / 10  # Escala 9-10
            elif 130 <= angle < 150:
                return 7 + (angle - 130) / 20 * 2  # Escala 7-8
            elif 110 <= angle < 130:
                return 4 + (angle - 110) / 20 * 2  # Escala 4-6
            elif angle < 110:
                return max(0, (angle - 100) / 10 * 3)  # Escala 0-3
            return 0

        return (score_angle(left_angle) + score_angle(right_angle)) / 2

    def calculate_contraction_score(self, left_angle, right_angle):
        """Calcula la puntuación de la contracción en el punto máximo."""
        def score_angle(angle):
            if 30 <= angle <= 40:
                return 9 + (40 - angle) / 10  # Escala 9-10
            elif 45 <= angle <= 55:
                return 7 + (55 - angle) / 10 * 2  # Escala 7-8
            elif 60 <= angle <= 70:
                return 4 + (70 - angle) / 10 * 2  # Escala 4-6
            elif angle > 75:
                return max(0, 3 - (angle - 75) / 10 * 3)  # Escala 0-3
            return 0

        return (score_angle(left_angle) + score_angle(right_angle)) / 2

    def detect_press_banca(self, landmarks, counter, velocity):
        if landmarks:
            # Puntos clave relevantes
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            
            # Ángulos de codos
            left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

            # Detectar estado y calcular puntuaciones según el estado
            if self.state == "up" and left_elbow_angle <= 70 and right_elbow_angle <= 70:
                # Punto máximo de flexión (contracción)
                self.puntuacion_bajo = self.calculate_contraction_score(left_elbow_angle, right_elbow_angle)
                self.state = "down"

            elif self.state == "down" and left_elbow_angle >= 135 and right_elbow_angle >= 135:
                # Punto máximo de extensión
                self.puntuacion_extension = self.calculate_extension_score(left_elbow_angle, right_elbow_angle)
                self.state = "up"
                
                
                # Calcular ROM solo al finalizar una repetición completa
                self.puntuacion_rom = np.mean([self.puntuacion_extension, self.puntuacion_bajo])
                self.ROM.append(self.puntuacion_rom)
            # Actualizar repetición al volver a la extensión
            last_move = counter.state
            self.repeticion = counter.update(left_elbow_angle, right_elbow_angle)
            if (counter.state == "up" and last_move == "down"):
                current_time = time.time()
                velocity.update(current_time)
        # Retornar siempre los valores actuales
        return [
            f"Extension: {self.puntuacion_extension:.2f}",
            f"Contraccion: {self.puntuacion_bajo:.2f}",
            f"ROM: {self.puntuacion_rom:.2f}",
            f"Repeticiones: {self.repeticion}",
            f"Press de banca, velocidad: {velocity.get_average_velocity():.2f} m/s"
        ]
            

# Cambia el argumento a la ruta de tu video
video_path = "press_banca2.mp4"  # Reemplaza con la ruta a tu archivo de video
cap = cv2.VideoCapture(video_path)
counter = PressBancaCounter()
velocity = VelocityCalculator()
state = PressBancaState()
RM = Rm(float(input("Introduce el peso con el que realizas el ejercicio: ")))
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Fin del video o archivo no encontrado.")
        
        break

    # Convierte la imagen a RGB (MediaPipe requiere RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb_frame.flags.writeable = False

    # Procesa la detección de poses
    results = pose.process(rgb_frame)

    # Visualiza los resultados
    rgb_frame.flags.writeable = True
    frame = cv2.cvtColor(rgb_frame, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )

        # Detecta el ejercicio basado en las posiciones
        exercise1, exercise2, exercise3, exercise4, exercise5 = state.detect_press_banca(results.pose_landmarks.landmark, counter, velocity)

        # Ajuste dinámico del tamaño del texto según el tamaño del frame
        height, width, _ = frame.shape
        font_scale = width / 950  # Escala el tamaño del texto en función del ancho del video
        font = cv2.FONT_HERSHEY_SIMPLEX

        # Ponemos el texto en la esquina superior izquierda
        cv2.putText(
            frame, exercise1, (10, 40), font, font_scale, (0, 255, 0), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise2, (10, 80), font, font_scale, (0, 255, 0), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise3, (10, 120), font, font_scale, (0, 255, 0), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise4, (10, 160), font, font_scale, (0, 255, 0), 2, cv2.LINE_AA
        )
        cv2.putText(
            frame, exercise5, (10, 200), font, font_scale, (0, 255, 0), 2, cv2.LINE_AA
        )

    # Muestra la salida
    cv2.imshow('Gym Exercise Analysis', frame)

    # Salir con 'q'
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
Epley = RM.Epley(exercise4)
Brzycki  = RM.Brzycki(exercise4)
EpleyReducido = RM.EpleyReducida(exercise4)
reps = RM.RepsToInt(exercise4)
ROM = state.ROM
print(f"repeticiones totales: {reps:.2f}")
print(f" RM con EPLEY: {Epley:.2f} \n", f"RM con BRZYCKI(bueno a repeticiones bajas): {Brzycki:.2f}\n", f"RM con EPLEYREDUCIDO: {EpleyReducido:.2f}\n", f"Media de ROM: {np.mean(ROM):.2f}\n", f"Velocidad media: {velocity.get_average_velocity():.2f} m/s")
cap.release()
cv2.destroyAllWindows()


d:\anaconda3\envs\VC_P5\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Fin del video o archivo no encontrado.
repeticiones totales: 8.00
 RM con EPLEY: 126.40 
 RM con BRZYCKI(bueno a repeticiones bajas): 124.16
 RM con EPLEYREDUCIDO: 122.40
 Media de ROM: 6.26
 Velocidad media: 0.24 m/s
